In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_bayesian_regret_variable_cost_runs(acq, dim):
    runs = api.runs(path="ziv-scully-group/Gittins for Bayesian Optimization", filters={
        "sweep": "0ii9yc4n", 
        "config.lengthscale": 0.01, 
        "config.amplitude": 1, 
        "config.kernel": "matern52", 
        "config.dim": dim,
        "config.problem": 'hard_for_eipc',
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
      'ExpectedImprovement':'ExpectedImprovementPerCost',
      'ExpectedImprovementWithCost_Uniform':'ExpectedImprovementPerCost_Uniform',
      'ExpectedImprovementWithCost_Cooling':'ExpectedImprovementPerCost_Cooling',
      'Gittins_Lambda_01':'Gittins_Lambda01',
      'Gittins_Lambda_001':'Gittins_Lambda001',
      'Gittins_Lambda_0001':'Gittins_Lambda0001',
      'Gittins_Step_Divide2':'Gittins_Step_Divide2',
      'Gittins_Step_Divide5':'Gittins_Step_Divide5',
      'Gittins_Step_Divide10':'Gittins_Step_Divide10',
      'Gittins_Step_EIpu':'Gittins_Step_EIPC',
    }
dimensions = [1]

In [3]:
grouped_runs = {(a,d): load_bayesian_regret_variable_cost_runs(a,d) for a in acquisition_functions.keys() for d in (dimensions)}

100%|███████████████████████████████████████████| 20/20 [00:22<00:00,  1.10s/it]


In [6]:
for a in acquisition_functions.keys():
    for d in dimensions:
        config_and_metrics_per_seed = grouped_runs[a,d]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed]).T
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed]).T
        global_optimum_per_seed = np.array([m['global optimum value'][0] for (_,m) in config_and_metrics_per_seed])

        regret_per_seed = global_optimum_per_seed - best_observed_per_seed

        regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
        regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
        regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)

        np.savetxt(f"results/variable_amplitude/VariableAmplitude_EIpc_d{d}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')